In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from wordcloud import WordCloud
from textwrap import wrap
import unicodedata


# "..\Data\Banco-Preguntas-Icfes.txt"

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\santi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\santi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
file_path = "../Data/Banco-Preguntas-Icfes.txt"

# Leer el archivo línea por línea
with open(file_path, "r", encoding="utf-8") as file:
    lines = file.readlines()

# Mostrar las primeras 10 líneas para verificar el formato
print("Contenido del archivo (primeras 10 líneas):\n")
for i, line in enumerate(lines[:10]):
    print(f"Línea {i+1}: {line.strip()}")


Contenido del archivo (primeras 10 líneas):

Línea 1: Pregunta: ¿En qué fechas se dará inicio al proceso de registro e inscripción a los exámenes?
Línea 2: Respuesta: Validar en la página www.icfes.gov.co / Evaluaciones y Elija el examen de su interés.
Línea 3: 
Línea 4: Pregunta:¿Cómo puedo cambiar el correo de notificaciones asociado al usuario PRISMA de mi sede jornada?
Línea 5: Respuesta:A través del Chat, ingresando a la  página www.icfes.gov.co / Atención y servicios a la ciudadanía / canales de atención / Chat.(Link:https://www.icfes.gov.co/atencion-y-servicios-a-la-ciudadania/canales-de-atencion/)
Línea 6: 
Línea 7: Pregunta:¿qué debo hacer para presentar las pruebas icfes por segunda vez?
Línea 8: Respuesta:Se puede registrar las veces que desee, después de la quinta inscripción la tarifa aumenta, validar en  la página www.icfes.gov.co / Evaluaciones y Elija el examen de su interés.
Línea 9: 
Línea 10: Pregunta:¿El examen saber 11°, entrega certificado o constancia de asistenc

In [3]:
# 📌 2️⃣ Inicializar listas para almacenar preguntas y respuestas
preguntas = []
respuestas = []
pregunta_actual = ""
respuesta_actual = ""
leyendo_respuesta = False

# 📌 3️⃣ Leer el archivo línea por línea y separar preguntas y respuestas
with open(file_path, "r", encoding="utf-8") as file:
    for line in file:
        line = line.strip()

        # Detectar preguntas
        if line.startswith("Pregunta:"):
            if pregunta_actual:
                preguntas.append(pregunta_actual.strip())
                respuestas.append(respuesta_actual.strip())
            pregunta_actual = line.replace("Pregunta:", "").strip()
            respuesta_actual = ""
            leyendo_respuesta = False

        # Detectar respuestas
        elif line.startswith("Respuesta:"):
            leyendo_respuesta = True
            respuesta_actual = line.replace("Respuesta:", "").strip()

        # Agregar contenido a la pregunta o respuesta
        else:
            if leyendo_respuesta:
                respuesta_actual += " " + line.strip()
            else:
                pregunta_actual += " " + line.strip()

# 📌 4️⃣ Agregar la última pregunta y respuesta
if pregunta_actual:
    preguntas.append(pregunta_actual.strip())
    respuestas.append(respuesta_actual.strip())

# 📌 5️⃣ Crear un DataFrame con preguntas y respuestas
df = pd.DataFrame({"Pregunta": preguntas, "Respuesta": respuestas})

# 📌 6️⃣ Mostrar las primeras filas para verificar los datos
print(df.head())

# 📌 7️⃣ Mostrar el número total de preguntas y respuestas extraídas
print(f"\n📊 Total de preguntas extraídas: {len(preguntas)}")
print(f"📊 Total de respuestas extraídas: {len(respuestas)}")


                                            Pregunta  \
0  ¿En qué fechas se dará inicio al proceso de re...   
1  ¿Cómo puedo cambiar el correo de notificacione...   
2  ¿qué debo hacer para presentar las pruebas icf...   
3  ¿El examen saber 11°, entrega certificado o co...   
4                       ¿Debo pagar las pruebas TyT?   

                                           Respuesta  
0  Validar en la página www.icfes.gov.co / Evalua...  
1  A través del Chat, ingresando a la  página www...  
2  Se puede registrar las veces que desee, despué...  
3  No, únicamente se genera certificado de asiste...  
4  Valida las tarifas en la página www.icfes.gov....  

📊 Total de preguntas extraídas: 26
📊 Total de respuestas extraídas: 26


In [4]:
# 📌 1️⃣ Función para eliminar tildes y caracteres especiales
def limpiar_texto(texto):
    texto = str(texto).lower()  # Convertir a minúsculas
    texto = re.sub(r'\W+', ' ', texto)  # Eliminar caracteres especiales (excepto letras y espacios)
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')  # Eliminar tildes
    return texto.strip()  # Eliminar espacios extra

# 📌 2️⃣ Aplicar la función a las columnas del DataFrame
df["Pregunta_Limpia"] = df["Pregunta"].apply(limpiar_texto)
df["Respuesta_Limpia"] = df["Respuesta"].apply(limpiar_texto)

# 📌 3️⃣ Mostrar las primeras filas para verificar los datos limpios
print(df[["Pregunta_Limpia", "Respuesta_Limpia"]].head())


                                     Pregunta_Limpia  \
0  en que fechas se dara inicio al proceso de reg...   
1  como puedo cambiar el correo de notificaciones...   
2  que debo hacer para presentar las pruebas icfe...   
3  el examen saber 11 entrega certificado o const...   
4                         debo pagar las pruebas tyt   

                                    Respuesta_Limpia  
0  validar en la pagina www icfes gov co evaluaci...  
1  a traves del chat ingresando a la pagina www i...  
2  se puede registrar las veces que desee despues...  
3  no unicamente se genera certificado de asisten...  
4  valida las tarifas en la pagina www icfes gov ...  


In [5]:
# 📌 2️⃣ Definir stopwords en español
stop_words = set(stopwords.words("spanish"))

# 📌 3️⃣ Función para tokenizar y eliminar stopwords
def tokenizar_y_filtrar(texto):
    tokens = word_tokenize(texto)  # Tokenizar el texto
    tokens = [word for word in tokens if word not in stop_words]  # Filtrar palabras vacías
    return tokens

# 📌 4️⃣ Tokenizar preguntas y respuestas limpias
tokens_preguntas = [token for pregunta in df["Pregunta_Limpia"] for token in tokenizar_y_filtrar(pregunta)]
tokens_respuestas = [token for respuesta in df["Respuesta_Limpia"] for token in tokenizar_y_filtrar(respuesta)]

# 📌 5️⃣ Contar frecuencia de palabras
frecuencia_palabras_preguntas = Counter(tokens_preguntas)
frecuencia_palabras_respuestas = Counter(tokens_respuestas)

# 📌 6️⃣ Obtener las 10 palabras más comunes en cada conjunto
palabras_comunes_preguntas = frecuencia_palabras_preguntas.most_common(10)
palabras_comunes_respuestas = frecuencia_palabras_respuestas.most_common(10)

# 📌 7️⃣ Mostrar los resultados
print("\n📊 Palabras más comunes en preguntas:")
for palabra, freq in palabras_comunes_preguntas:
    print(f"{palabra}: {freq}")

print("\n📊 Palabras más comunes en respuestas:")
for palabra, freq in palabras_comunes_respuestas:
    print(f"{palabra}: {freq}")

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\santi/nltk_data'
    - 'c:\\Users\\santi\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\Local\\pypoetry\\Cache\\virtualenvs\\icfes_pro-tNgj2KN3-py3.12\\nltk_data'
    - 'c:\\Users\\santi\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\Local\\pypoetry\\Cache\\virtualenvs\\icfes_pro-tNgj2KN3-py3.12\\share\\nltk_data'
    - 'c:\\Users\\santi\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\Local\\pypoetry\\Cache\\virtualenvs\\icfes_pro-tNgj2KN3-py3.12\\lib\\nltk_data'
 